In [1]:
from bilstm_crf.model import BiLstmCRF
import tensorflow as tf
import json
import tensorflow_addons as tfa

/home/dtadmin/miniconda3/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:54: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.3.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [2]:
dropout_rate = 0.5
embedding_size = 32
hidden_dim = 32

In [3]:
with open('/data/share/user/bruce.zhang/NER/conll2003/vocab.json', 'r', encoding='utf-8') as f:
    vocab = json.load(f)
    
with open('/data/share/user/bruce.zhang/NER/conll2003/label.json', 'r', encoding='utf-8') as f:
    labels = json.load(f)

In [4]:
id2label = {v: k for k,v in labels.items()}

In [5]:
vocab_size = len(vocab)

In [6]:
model = BiLstmCRF(vocab_size, embedding_size, hidden_dim,len(labels), 0.5)

In [7]:
model([tf.constant([[1,2,3]]), tf.constant([[2,3,4]])])

(<tf.Tensor: shape=(1, 3, 8), dtype=float32, numpy=
 array([[[-0.01030804,  0.00653901, -0.00562507,  0.00066606,
          -0.00342177, -0.00071182,  0.00478643,  0.008395  ],
         [-0.00492124,  0.00758045, -0.00582938,  0.00053183,
          -0.00439965, -0.00619917,  0.01543278,  0.00068343],
         [-0.00011463,  0.00485278,  0.00187784, -0.00239379,
          -0.00595655, -0.00500973,  0.00796062,  0.00840901]]],
       dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=int32, numpy=array([3], dtype=int32)>)

In [8]:
model.load_weights('/data/share/user/bruce.zhang/NER/model/model.ckpt-7')

In [27]:
# sent = 'SOCCER - Japan GET LUCKY WIN , China IN SURPRISE DEFEAT .'
sent = 'despite winning the Asian Games title two years ago , Uzbekistan are in the finals as outsiders .'
w_idx = [vocab.get(w, 1) for w in sent.lower().split(' ')]
print(w_idx)

[779, 2854, 15, 5611, 1837, 1973, 393, 1199, 2575, 70, 1, 170, 230, 15, 3121, 142, 12223, 10]


In [23]:
logits, text_lens = model(tf.constant([w_idx]), training=False)

In [24]:
viterbi_path, _  = tfa.text.viterbi_decode(logits[0], model.transition_params)

In [25]:
viterbi_path = [id2label[i] for i in viterbi_path]
viterbi_path

['O',
 'O',
 'O',
 'I-MISC',
 'I-MISC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']